In [2]:
use std::env;
use std::path::Path;

fn main() -> std::io::Result<()> {
    let path = env::current_dir()?;
    println!("The current directory is {}", path.display());
    Ok(())
}
main();

let path = Path::new("/home/bzhan/projects/postflop-solver");
assert!(env::set_current_dir(&path).is_ok());
println!("Successfully changed working directory to {}!", path.display());

The current directory is /home/bzhan/projects/postflop-solver


Successfully changed working directory to /home/bzhan/projects/postflop-solver!


In [ ]:
    /// Builds the action tree.
    #[inline]
    fn build_tree(&mut self) {
        
        let mut root = self.root.lock();
        let curr_config = self.config();
        *root = StudActionTreeNode::default();
        root.board_state = curr_config.initial_state;

        // CR-someday: if we implement all ins make sure to pass in the new
        self.build_tree_recursive(&mut root, StudBuildTreeInfo::new(curr_config.small_bet_size));

    }

    #[inline]
    fn get_bet_size(node: &StudBoardState, curr_config: &StudTreeConfig) -> i32 {
        match node {
            StudBoardState::ThirdStreet | StudBoardState::FourthStreet => curr_config.small_bet_size,
            StudBoardState::FifthStreet | StudBoardState::SixthStreet | StudBoardState::SeventhStreet => curr_config.big_bet_size,
        }
    }

    /// Recursively builds the action tree.
    fn build_tree_recursive(&self, node: &mut StudActionTreeNode, info: StudBuildTreeInfo) {
        let curr_config: &StudTreeConfig = self.config();

        if node.is_terminal() {
            // do nothing
        } else if node.is_chance() {
            let next_state = match node.board_state {
                StudBoardState::ThirdStreet => StudBoardState::FourthStreet,
                StudBoardState::FourthStreet => StudBoardState::FifthStreet,
                StudBoardState::FifthStreet => StudBoardState::SixthStreet,
                StudBoardState::SixthStreet => StudBoardState::SeventhStreet,
                StudBoardState::SeventhStreet => unreachable!(),
            };
            let bet_size_on_this_street = StudActionTree::get_bet_size(&next_state, curr_config);

            // <TODO> calculate new player, but need the chance cards... maybe search through self.history
            // let next_player = StudActionTreeNode::calc_next_player(&node, )
            let _next_player = PLAYER_OOP;

            // Cr-someday: this is another spot to think about all-ins
            node.actions_so_far_this_game.push(StudAction::Chance((0,0)));
            
            //<TODO> why can we just push default values onto StudActionTreeNode for the last two variables???
            node.children.push(MutexLike::new(StudActionTreeNode {
                player_whose_action: PLAYER_OOP,
                board_state: next_state,
                amount_wagered_so_far_this_game: node.amount_wagered_so_far_this_game,
                ..Default::default()
            }));

            self.build_tree_recursive(
                &mut node.children[0].lock(),
                info.create_next(PLAYER_OOP, StudAction::Chance((0,0)), bet_size_on_this_street),
            );
        } else {
            self.push_actions(node, &info);
            let bet_size_on_this_street = StudActionTree::get_bet_size(&node.board_state, curr_config);
            for (action, child) in node.actions_so_far_this_game.iter().zip(node.children.iter()) {
                self.build_tree_recursive(
                    &mut child.lock(),
                    info.create_next(node.player_whose_action, *action, bet_size_on_this_street),
                );
            }
        }
    }

    /// Pushes all possible actions to the given node.
    fn push_actions(&self, node: &mut StudActionTreeNode, info: &StudBuildTreeInfo) {
        let mut actions = Vec::new();
        let player = node.player_whose_action;
        let opponent = node.player_whose_action ^ 1;
        let curr_config: &StudTreeConfig = self.config();

        match info.prev_action {
            StudAction::Chance(_) | StudAction::None => {
                actions.push(StudAction::Check);
                actions.push(StudAction::Bet(info.bet_size_on_this_street));
            }
            StudAction::Check => {
                actions.push(StudAction::Check);     
                actions.push(StudAction::Bet(info.bet_size_on_this_street));
            }
            StudAction::Bet(_) | StudAction::Raise(_) => {
                actions.push(StudAction::Fold);
                actions.push(StudAction::Call);

                if info.num_bets_on_this_street < curr_config.bets_cap { 
                    actions.push(StudAction::Raise(info.bet_size_on_this_street * (info.num_bets_on_this_street + 1))); 
                }
            }
            StudAction::Call => {
            }
            StudAction::Fold => {
            }
        }

        // remove duplicates
        actions.sort_unstable();
        actions.dedup();

        let player_after_call = match node.board_state {
            StudBoardState::SeventhStreet => PLAYER_TERMINAL_FLAG,
            _ => PLAYER_CHANCE_FLAG | player,
        };

        let player_after_check = match player {
            PLAYER_OOP => opponent,
            _ => player_after_call,
        };

        // push actions
        for action in actions {
            let mut amount = node.amount_wagered_so_far_this_game;
            let next_player = match action {
                StudAction::Fold => PLAYER_FOLD_FLAG | player,
                StudAction::Check => player_after_check,
                StudAction::Call => {
                    amount += info.bet_size_on_this_street;
                    player_after_call
                }
                StudAction::Bet(_) | StudAction::Raise(_) => {
                    amount += info.bet_size_on_this_street;
                    opponent
                }
                _ => panic!("Unexpected action: {action:?}"),
            };

            node.actions_so_far_this_game.push(action);
            node.children.push(MutexLike::new(StudActionTreeNode {
                player_whose_action: next_player,
                board_state: node.board_state,
                amount_wagered_so_far_this_game: amount,
                ..Default::default()
            }));
        }

        node.actions_so_far_this_game.shrink_to_fit();
        node.children.shrink_to_fit();
    }

In [ ]:
impl StudBuildTreeInfo {
    #[inline]
    fn new(bet_size: i32) -> Self {
        Self {
            prev_action: StudAction::None,
            num_bets_on_this_street: 0,
            bet_size_on_this_street:bet_size,
        }
    }

    #[inline]
    // Don't really need player for now if we don't have all-in considerations
    fn create_next(&self, _player: u8, action: StudAction, bet_size_on_this_street: i32) -> Self {
        let mut num_bets_on_this_street = self.num_bets_on_this_street;
        
        match action {
            StudAction::Check => {
            }
            StudAction::Call => {
                num_bets_on_this_street = 0;
            }
            StudAction::Bet(_) => {
                num_bets_on_this_street += 1;
            }
            StudAction::Raise(_) => {
                num_bets_on_this_street += 1;
            }
            _ => {}
        }

        // bet_size_on_this_street only encodes whether we are in small or big bet world
        // the actual bet size can be considered num_bets_on_this_street * bet_size_on_this_street
        StudBuildTreeInfo {
            prev_action: action,
            num_bets_on_this_street,
            bet_size_on_this_street, 
        }
    }
}

/// Returns the number of action nodes of [flop, turn, river].
#[allow(dead_code)]
pub(crate) fn count_num_action_nodes(node: &StudActionTreeNode) -> [u64; 3] {
    let mut ret = [0, 0, 0];
    count_num_action_nodes_recursive(node, 0, &mut ret);
    if ret[1] == 0 {
        ret = [0, 0, ret[0]];
    } else if ret[2] == 0 {
        ret = [0, ret[0], ret[1]];
    }
    ret
}

#[allow(dead_code)]
fn count_num_action_nodes_recursive(node: &StudActionTreeNode, street: usize, count: &mut [u64; 3]) {
    count[street] += 1;
    if node.is_terminal() {
        // do nothing
    } else if node.is_chance() {
        count_num_action_nodes_recursive(&node.children[0].lock(), street + 1, count);
    } else {
        for child in &node.children {
            count_num_action_nodes_recursive(&child.lock(), street, count);
        }
    }
}


In [2]:
:dep postflop-solver = { path = "/home/bzhan/projects/postflop-solver" }
use postflop_solver::*;

In [21]:
let mut tree_config = StudTreeConfig {
    initial_state: StudBoardState::SixthStreet,
    starting_pot_from_antes: 10,
    effective_stack: 100,
    bets_cap: 4,
    small_bet_size: 2,
    big_bet_size: 4,
};

Error: variable does not need to be mutable

Error: constant `PLAYER_OOP` is never used

Error: constant `PLAYER_FOLD_FLAG` is never used

Error: struct `StudBuildTreeInfo` is never constructed

Error: 4 warnings emitted

Error: borrowed data escapes outside of function

In [ ]:
let action_tree = StudActionTree::new(tree_config).unwrap();

In [9]:
use polars::prelude::*;
use std::collections::HashMap;

fn main() -> PostFlopGame { //Result<(DataFrame, DataFrame)> {
    // ranges of OOP and IP in string format
    // see the documentation of `Range` for more details about the format
    let oop_range = "66+,A8s+,A5s-A4s,AJo+,K9s+,KQo,QTs+,JTs,96s+,85s+,75s+,65s,54s";
    let ip_range = "QQ-22,AQs-A2s,ATo+,K5s+,KJo+,Q8s+,J8s+,T7s+,96s+,86s+,75s+,64s+,53s+";

    let card_config = CardConfig {
        range: [oop_range.parse().unwrap(), ip_range.parse().unwrap()],
        flop: flop_from_str("Td9d6h").unwrap(),
        turn: card_from_str("Qc").unwrap(),
        river: NOT_DEALT,
    };

    // bet sizes -> 60% of the pot, geometric size, and all-in
    // raise sizes -> 2.5x of the previous bet
    // see the documentation of `BetSizeOptions` for more details
    let bet_sizes = BetSizeOptions::try_from(("60%, e, a", "2.5x")).unwrap();

    let tree_config = TreeConfig {
        initial_state: BoardState::Turn, // must match `card_config`
        starting_pot: 200,
        effective_stack: 900,
        rake_rate: 0.0,
        rake_cap: 0.0,
        flop_bet_sizes: [bet_sizes.clone(), bet_sizes.clone()], // [OOP, IP]
        turn_bet_sizes: [bet_sizes.clone(), bet_sizes.clone()],
        river_bet_sizes: [bet_sizes.clone(), bet_sizes],
        turn_donk_sizes: None, // use default bet sizes
        river_donk_sizes: Some(DonkSizeOptions::try_from("50%").unwrap()),
        add_allin_threshold: 1.5, // add all-in if (maximum bet size) <= 1.5x pot
        force_allin_threshold: 0.15, // force all-in if (SPR after the opponent's call) <= 0.15
        merging_threshold: 0.1,
    };

    // build the game tree
    // `ActionTree` can be edited manually after construction
    let action_tree = ActionTree::new(tree_config).unwrap();
    let mut game = PostFlopGame::with_config(card_config, action_tree).unwrap();

    // obtain the private hands
    let oop_cards: &[(u8, u8)] = game.private_cards(0);
    let oop_cards_str = holes_to_strings(oop_cards).unwrap();
    println!("{:?}", oop_cards_str);

    let ip_cards = game.private_cards(1);
    let ip_cards_str = holes_to_strings(ip_cards).unwrap();
    println!("{:?}", ip_cards_str);
    
    let (mem_usage, mem_usage_compressed) = game.memory_usage();
    println!(
        "Memory usage without compression (32-bit float): {:.2}GB",
        mem_usage as f64 / (1024.0 * 1024.0 * 1024.0)
    );
    println!(
        "Memory usage with compression (16-bit integer): {:.2}GB",
        mem_usage_compressed as f64 / (1024.0 * 1024.0 * 1024.0)
    );

    game.allocate_memory(false);

    let max_num_iterations = 1000;
    let target_exploitability = game.tree_config().starting_pot as f32 * 0.005; // 0.5% of the pot
    println!("Target exploitability: {:.2}", target_exploitability);
    //let exploitability = solve(&mut game, max_num_iterations, target_exploitability, true);
    //println!("Exploitability: {:.2}", exploitability);
    
    let mut last_exploit = 1000.0;
    for i in 0..max_num_iterations {
        solve_step(&game, i);
        if (i + 1) % 100 == 0 {
            let exploitability = compute_exploitability(&game);
            println!("Exploitability on iteration {}: {:.2}", i, exploitability);
            if ((last_exploit - exploitability) < 0.01) {
                break
            }
            last_exploit=exploitability;
        }
     }
    finalize(&mut game);
    game.cache_normalized_weights();
    game
    }

In [10]:
bet_sizes

BetSizeOptions { bet: [PotRelative(0.6), Geometric(0, inf), AllIn], raise: [PrevBetRelative(2.5)] }